In [1]:
from sentinel_tiles import sentinel_tiles
import global_forecasting_system
from global_forecasting_system import forecast_Ta_C

In [2]:
time_UTC = "2025-01-12 00:00:00"

In [3]:
geometry = sentinel_tiles.grid("11SPS")
geometry

{
  "dimensions": {
    "rows": 3660,
    "cols": 3660
  },
  "bbox": {
    "xmin": 600000.0,
    "ymin": 3590220.0,
    "xmax": 709800.0,
    "ymax": 3700020.0
  },
  "crs": "EPSG:32611",
  "resolution": {
    "cell_width": 30.0,
    "cell_height": -30.0
  }
}

In [4]:
from global_forecasting_system import earliest_time_UTC

earliest = earliest_time_UTC()
earliest

[2026-01-12 09:34:46 INFO] getting GFS listing from date URL: https://www.ncei.noaa.gov/data/global-forecast-system/access/grid-004-0.5-degree/forecast/202601/20260110


datetime.datetime(2026, 1, 10, 0, 0)

In [5]:
# Directly call the date() method on the earliest object
# No need for to_pydatetime() since earliest is already a datetime object
earliest.date()

datetime.date(2026, 1, 10)

In [6]:
type(earliest)

datetime.datetime

In [7]:
listing = global_forecasting_system.get_GFS_listing()
listing

[2026-01-12 09:34:51 INFO] getting GFS listing from date URL: https://www.ncei.noaa.gov/data/global-forecast-system/access/grid-004-0.5-degree/forecast/202601/20260110


,forecast_time_UTC,address
0,2026-01-10 00:00:00,https://www.ncei.noaa.gov/data/global-forecast...
1,2026-01-10 03:00:00,https://www.ncei.noaa.gov/data/global-forecast...
129,2026-01-10 06:00:00,https://www.ncei.noaa.gov/data/global-forecast...
130,2026-01-10 09:00:00,https://www.ncei.noaa.gov/data/global-forecast...
258,2026-01-10 12:00:00,https://www.ncei.noaa.gov/data/global-forecast...
...,...,...
382,2026-01-26 00:00:00,https://www.ncei.noaa.gov/data/global-forecast...
383,2026-01-26 03:00:00,https://www.ncei.noaa.gov/data/global-forecast...
384,2026-01-26 06:00:00,https://www.ncei.noaa.gov/data/global-forecast...
385,2026-01-26 09:00:00,https://www.ncei.noaa.gov/data/global-forecast...


In [8]:
Ta_C = forecast_Ta_C(
    time_UTC=time_UTC,
    geometry=geometry,
)

Ta_C

[2026-01-12 09:34:56 INFO] getting GFS listing from date URL: https://www.ncei.noaa.gov/data/global-forecast-system/access/grid-004-0.5-degree/forecast/202601/20260110


ValueError: no GFS files found before time 2025-01-12 00:00:00 UTC

In [ ]:
listing[listing.forecast_time_UTC.apply(lambda forecast_time_UTC: str(forecast_time_UTC) <= str(time_UTC))]

In [ ]:
before = listing[listing.forecast_time_UTC.apply(lambda forecast_time_UTC: str(forecast_time_UTC) <= str(time_UTC))].iloc[[-1]]

In [ ]:
# Let's debug the issue by examining the data
from global_forecasting_system import GFS_file_addresses
import pandas as pd
import posixpath
from datetime import datetime
from os.path import splitext

# Get some sample data to see the issue
try:
    addresses = GFS_file_addresses()[:10]  # Just get first 10 for debugging
    address_df = pd.DataFrame({"address": addresses})
    address_df["basename"] = address_df.address.apply(lambda address: posixpath.basename(address))
    print("Sample basenames:")
    print(address_df["basename"].head())
    
    print("\nTrying to extract source_hour:")
    address_df["source_hour"] = address_df.basename.apply(lambda basename: int(basename.split("_")[3][:2]))
    print("Source hours:")
    print(address_df["source_hour"])
    print("Source hour dtypes:")
    print(address_df["source_hour"].dtype)
    
except Exception as e:
    print(f"Error during debugging: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Let's test the GFS most recent date address function
from global_forecasting_system import GFS_most_recent_date_address

try:
    date_url = GFS_most_recent_date_address()
    print(f"Date URL: {date_url}")
    
    # Test if we can fetch the page
    import requests
    response = requests.get(date_url)
    print(f"Response status: {response.status_code}")
    print(f"Response length: {len(response.text)}")
    
    if response.status_code == 200:
        from bs4 import BeautifulSoup
        soup = BeautifulSoup(response.text, "html.parser")
        links = [link.get("href") for link in soup.find_all("a")]
        print(f"Total links found: {len(links)}")
        
        # Look for .grb2 files
        from fnmatch import fnmatch
        grb2_files = [link for link in links if fnmatch(link, "*.grb2")]
        print(f"GRB2 files found: {len(grb2_files)}")
        if len(grb2_files) > 0:
            print("Sample GRB2 files:")
            for i, file in enumerate(grb2_files[:5]):
                print(f"  {i+1}: {file}")
        
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Reload the module to get the latest changes
import importlib
import global_forecasting_system
importlib.reload(global_forecasting_system)

# Test the updated function
from global_forecasting_system import earliest_time_UTC

try:
    earliest = earliest_time_UTC()
    print(f"Success: {earliest}")
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Test again with fresh import
from global_forecasting_system import get_GFS_listing

try:
    listing = get_GFS_listing()
    print(f"GFS listing length: {len(listing)}")
    print(f"Columns: {listing.columns.tolist()}")
    print(f"Dtypes: {listing.dtypes}")
    
    if len(listing) == 0:
        print("Empty listing returned successfully!")
    else:
        print("Non-empty listing:")
        print(listing.head())
        
except Exception as e:
    print(f"Error in get_GFS_listing: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Test the earliest_time_UTC function
from global_forecasting_system import earliest_time_UTC

try:
    earliest = earliest_time_UTC()
    print(f"Success: {earliest}")
except ValueError as e:
    print(f"Expected ValueError: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")
    import traceback
    traceback.print_exc()